In [104]:
from typing import Final
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

In [105]:
df = pd.read_excel('historyIndex.xls')
df = df.iloc[6:647].copy()

In [106]:
df

,Unnamed: 0,Unnamed: 1
6,"Dec 31, 1969",100.000
7,"Jan 30, 1970",94.455
8,"Feb 27, 1970",97.405
9,"Mar 31, 1970",97.708
10,"Apr 30, 1970",88.578
...,...,...
642,"Dec 30, 2022","7,985.857"
643,"Jan 31, 2023","8,550.905"
644,"Feb 28, 2023","8,345.354"
645,"Mar 31, 2023","8,603.223"


In [107]:
df.reset_index(drop=True, inplace=True)
df.columns = ['date', 'price']

In [108]:
df['date'] = pd.to_datetime(df['date'])
df = df.replace(',','', regex=True)
df['price'] = df['price'].astype(float)

In [109]:
df['1m_return'] = (df['price']/df['price'].shift(1) - 1)
df['3m_return'] = (df['price']/df['price'].shift(3) - 1)
df['6m_return'] = (df['price']/df['price'].shift(6) - 1)
df['12m_return'] = (df['price']/df['price'].shift(12) - 1)
df['2y_return'] = (df['price']/df['price'].shift(24) - 1)
df['3y_return'] = (df['price']/df['price'].shift(36) - 1)
df['5y_return'] = (df['price']/df['price'].shift(60) - 1)
df['10y_return'] = (df['price']/df['price'].shift(120) - 1)
df['20y_return'] = (df['price']/df['price'].shift(240) - 1)

In [110]:
df['1m_annualized'] = (1 + df['1m_return'])**12 - 1
df['3m_annualized'] = (1 + df['3m_return'])**4 - 1
df['6m_annualized'] = (1 + df['6m_return'])**2 - 1
df['12m_annualized'] = (1 + df['12m_return'])**1 - 1
df['2y_annualized'] = (1 + df['2y_return'])**(1/2) - 1
df['3y_annualized'] = (1 + df['3y_return'])**(1/3) - 1
df['5y_annualized'] = (1 + df['5y_return'])**(1/5) - 1
df['10y_annualized'] = (1 + df['10y_return'])**(1/10) - 1
df['20y_annualized'] = (1 + df['20y_return'])**(1/20) - 1

In [111]:
df

,date,price,1m_return,3m_return,6m_return,12m_return,2y_return,3y_return,5y_return,10y_return,20y_return,1m_annualized,3m_annualized,6m_annualized,12m_annualized,2y_annualized,3y_annualized,5y_annualized,10y_annualized,20y_annualized
0,1969-12-31,100.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1970-01-30,94.455,-0.055450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.495688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1970-02-27,97.405,0.031232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.446357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1970-03-31,97.708,0.003111,-0.022920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.037974,-0.088576,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970-04-30,88.578,-0.093442,-0.062220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.691859,-0.226601,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,2022-12-30,7985.857,-0.042467,0.097655,0.029736,-0.181416,-0.002823,0.155753,0.347008,1.335756,3.817618,-0.405922,0.451653,0.060356,-0.181416,-0.001413,0.049434,0.061388,0.088536,0.081787
637,2023-01-31,8550.905,0.070756,0.096572,0.021493,-0.074529,0.078451,0.245108,0.369981,1.379799,4.320635,1.271363,0.445937,0.043448,-0.074529,0.038485,0.075810,0.064984,0.090571,0.087172
638,2023-02-28,8345.354,-0.024039,0.000638,0.040436,-0.073339,0.026230,0.327374,0.394833,1.318763,4.285235,-0.253221,0.002553,0.082508,-0.073339,0.013030,0.099000,0.068820,0.087741,0.086809
639,2023-03-31,8603.223,0.030900,0.077307,0.182512,-0.070222,0.023874,0.577108,0.469974,1.335702,4.466591,0.440778,0.346972,0.398333,-0.070222,0.011867,0.164002,0.080095,0.088533,0.088644


In [112]:
df.describe()

,price,1m_return,3m_return,6m_return,12m_return,2y_return,3y_return,5y_return,10y_return,20y_return,1m_annualized,3m_annualized,6m_annualized,12m_annualized,2y_annualized,3y_annualized,5y_annualized,10y_annualized,20y_annualized
count,641.000000,640.000000,638.000000,635.000000,629.000000,617.000000,605.000000,581.000000,521.000000,401.000000,640.000000,638.000000,635.000000,629.000000,617.000000,605.000000,581.000000,521.000000,401.000000
mean,2177.928928,0.007950,0.024297,0.050096,0.103908,0.221399,0.342989,0.633083,1.674218,5.888702,0.229787,0.137624,0.115990,0.103908,0.098253,0.094491,0.092703,0.094257,0.092784
std,2255.920846,0.043111,0.077279,0.115367,0.171728,0.272065,0.364172,0.598426,1.181182,3.914582,0.613680,0.330275,0.240305,0.171728,0.123545,0.098524,0.074351,0.047054,0.031623
min,80.946000,-0.189601,-0.332204,-0.435526,-0.471197,-0.474002,-0.456299,-0.253635,-0.227533,0.884585,-0.919761,-0.801127,-0.681369,-0.471197,-0.274743,-0.183818,-0.056829,-0.025486,0.032193
25%,259.050000,-0.016661,-0.013196,-0.014298,0.006793,0.081242,0.157271,0.205998,0.876256,2.537849,-0.182593,-0.051749,-0.028392,0.006793,0.039828,0.049893,0.038172,0.064950,0.065214
50%,1503.506000,0.011799,0.027400,0.053283,0.120567,0.241454,0.319956,0.567995,1.335702,4.721031,0.151143,0.114188,0.109405,0.120567,0.114206,0.096949,0.094130,0.088533,0.091123
75%,3166.489000,0.032601,0.070572,0.112761,0.200203,0.363302,0.519987,0.861720,2.285484,8.804882,0.469564,0.313600,0.238236,0.200203,0.167605,0.149776,0.132356,0.126315,0.120914
max,9755.694000,0.145823,0.304416,0.466621,0.659709,1.372536,1.969380,3.263036,5.219628,16.582382,4.121682,1.895109,1.150978,0.659709,0.540304,0.437326,0.336423,0.200539,0.154128


In [113]:
data = [
    go.Scatter(
        x=df['date'],
        y=df[column],
        name=column,
        mode='lines'
        )
    for column in df.columns[11:]
    ]

In [121]:
data = [
    go.Box(
        x=df[column],
        name=column,
        opacity=0.75
        )
    for column in df.columns[11:]
    ]

In [122]:
layout = go.Layout(
    barmode='overlay',
    bargap=0
)

In [123]:
go.Figure(data=data, layout=layout)

In [117]:
px.histogram(df, x='1m_return', nbins=100)

In [118]:
1.08746779258396264**(1/12)

1.0070121262571048

In [119]:
total_return = df.loc[640, 'price']/df.loc[0, 'price']

In [120]:
total_return**(1/640)-1

0.007012126257104789